<h1><span style="color:red">SuAVE Transfer Learning Model</span></h1>


For a simple Python implementation see https://www.pyimagesearch.com/2016/08/01/lenet-convolutional-neural-network-in-python/

The model was originally described in http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf

In [ ]:
#! pip install imutils

## 1. Disable autoscroll and retrieve survey parameters from the URL

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

## 2. Import all packages (this might take a few seconds)

In [3]:
# Import widget functionality
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
# import the necessary packages
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K
# import local lenet.py file describing the LeNet implementation with RELU activation functions
#from lenet import LeNet

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

# import the necessary packages for SVM predictor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import imutils


# copy from sample
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
# for vgg
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Dense, Dropout, Flatten, Input, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import optimizers
# mobilenet
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

def printmd(string):
    display(Markdown(string))


Using TensorFlow backend.


## 3. Initializing the number of times the model will loop and its batch size for learning

In [4]:
from collections import OrderedDict

epochCount = OrderedDict()
epochCount['5 Iterations'] = 5
epochCount['15 Iterations'] = 15
epochCount['25 Iterations'] = 25

def f(epoch_count):
    return epoch_count

epochNum = interact(f, epoch_count=epochCount)


interactive(children=(Dropdown(description='epoch_count', options=OrderedDict([('5 Iterations', 5), ('15 Itera…

In [5]:
batchS = OrderedDict()
batchS['32 Batch Size'] = 32
batchS['64 Batch Size'] = 64
batchS['128 Batch Size'] = 128

def f(batch_size):
    return batch_size

batchNum = interact(f, batch_size=batchS)

interactive(children=(Dropdown(description='batch_size', options=OrderedDict([('32 Batch Size', 32), ('64 Batc…

In [6]:
# init the number of epochs to train for, init learning rate and batch size
EPOCHS = epochNum.widget.result
INIT_LR = 1e-3
BS = batchNum.widget.result
# init the image suffix, yset, and image list
suffix = '.png'
img_list = []
yset = []
# create labels list and 2 dicts for 2 way mapping
labels = []
# key = label value = number
label_yval = dict()
# key = number value = label
yval_label = dict()

## 4. Load the data and choose the column name to be predicted

In [7]:
# use csv file to grab images/labels
absolutePath = "../../temp_csvs/"
# read the csv file
file = open(absolutePath + csv_file, encoding="latin-1")
df = pd.read_csv(file)

#generate image path
localdzc = dzc_file.replace("https://maxim.ucsd.edu/dzgen/lib-staging-uploads","/lib-nfs/dzgen")
full_images_location = localdzc.replace("/content.dzc","/full_images/")


# Choose column of label for prediction
toPredict = list(df.columns.values)

pred_menu = OrderedDict()
for i in range(0, len(toPredict)):
    pred_menu[toPredict[i]] = toPredict[i]

def f(predictions_menu):
    return predictions_menu
# choose which label for predictions
out2 = interact(f, predictions_menu=pred_menu)

interactive(children=(Dropdown(description='predictions_menu', options=OrderedDict([('PPL/XP', 'PPL/XP'), ('An…

## 5. Select pixel resolution

In [8]:
a = widgets.IntSlider(value=60,min=20,max=300,step=10,description='Size, pixels:')
display(a)

IntSlider(value=60, description='Size, pixels:', max=300, min=20, step=10)

In [9]:
printmd("<h2><span style='color:red'>Verify model parameters:</span></h2>")
printmd("<b>Variable to predict:  " +out2.widget.result + "</b>")
printmd("<b>Pixel resolution:  " +str(a.value) + "</b>")
printmd("<b>Path to images:  " +full_images_location + "</b>")
printmd("<b>Number of epochs:  " + str(EPOCHS) + "</b>")
printmd("<b>Batch size:  " +str(BS) + "</b>")


<h2><span style='color:red'>Verify model parameters:</span></h2>

<b>Variable to predict:  Petrographic Fabric</b>

<b>Pixel resolution:  60</b>

<b>Path to images:  /lib-nfs/dzgen/4d862392cf30aa7d520443c52b0599d2/full_images/</b>

<b>Number of epochs:  5</b>

<b>Batch size:  32</b>

## 6. Grab the images and configure them for predicting

### This might take a little while depending on the size of the dataset

In [10]:
im_dimension = a.value
# grab chosen column names
nameCol = df['#img']
predCol = df[out2.widget.result]

def matchImage(curr_image_array, image_list):
    for i in range(0, len(image_list)):
        if (np.array_equal(curr_image_array, image_list[i])):
            return i
    
labels = []
# add all fabric columns to the y set
for i in range (0,len(predCol)):
    labels.append(predCol[i])

# grab all unique labels
uni_labels = set(labels)
uni_labels = list(uni_labels)

# assign each label a dict key number
for i in range(0,len(uni_labels)):
    yval_label[i] = uni_labels[i]
    label_yval[uni_labels[i]] = i


yset = []    
# create list of keys associated with their labels
for i in range (0, len(labels)):
    yset.append(label_yval[labels[i]])

file_lst=[]

counter = 0
im_count = widgets.Label(value="0% images loaded")
display(im_count)
numfiles = len(nameCol)

# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]

    fileName = os.path.join(full_images_location, base_filename + suffix)
    
    file_lst.append(fileName)
    
    
    counter += 1
    im_count.value = str(int(counter / numfiles * 100)) + "% images loaded"

ndf=pd.DataFrame()
ndf['id']=file_lst
ndf['label']=list(map(str, yset))


Label(value='0% images loaded')

In [11]:
# if classes >=5
if len(np.unique(ndf['label']))>=5:
    main_class=ndf.groupby('label').count().reset_index().sort_values('id',ascending=False).reset_index(drop=True).loc[:4,'label'].to_list()
else:
    main_class=list(np.unique(ndf['label']))

In [12]:
main_class

['21', '17', '1', '7', '20']

In [13]:
empty=pd.DataFrame(columns=['id','label'])
for i in main_class:
    n=ndf.loc[ndf['label']==i]
    empty=pd.concat([empty,n])
empty = empty.sample(frac=1).reset_index(drop=True)

In [14]:
ndf.shape

(352, 2)

In [15]:
empty.shape

(220, 2)

In [16]:
datagen = ImageDataGenerator(rescale=1/255., validation_split=0.25)

train_generator = datagen.flow_from_dataframe(dataframe=empty, 
                                             x_col='id',
                                             y_col='label',
                                             target_size=(224,224),
                                             classes=main_class,
                                              subset='training',
                                                 color_mode='rgb',
                                                 batch_size=BS,
                                                 class_mode='categorical',
                                              
                                                 shuffle=True)
 
validation_generator = datagen.flow_from_dataframe(dataframe=empty, 
                                             x_col='id',
                                             y_col='label',
                                             target_size=(224,224),
                                             classes= main_class,
                                                subset='validation',
                                                 color_mode='rgb',
                                                 batch_size=BS,
                                                 class_mode='categorical',
                                                 shuffle=True)


Found 165 validated image filenames belonging to 5 classes.
Found 55 validated image filenames belonging to 5 classes.


In [17]:
num_classes = len(train_generator.class_indices)
train_steps = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

### trytry

In [18]:
#resnet
base_model=ResNet50(weights='imagenet',include_top=False)

newOutput=base_model.output
newOutput=GlobalAveragePooling2D()(newOutput)
newOutput=Dense(1024,activation='relu')(newOutput) 
newOutput=Dense(1024,activation='relu')(newOutput) 
newOutput=Dense(512,activation='relu')(newOutput)

preds=Dense(num_classes,activation='softmax')(newOutput)


model=Model(inputs=base_model.input,outputs=preds)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss='binary_crossentropy',optimizer = opt,metrics=['accuracy'])

#history = model.fit_generator(train_generator,steps_per_epoch=train_steps, epochs=5,
                              

In [ ]:
history = model.fit_generator(train_generator,steps_per_epoch=train_steps,
                              validation_data= validation_generator, 
                              validation_steps=validation_steps, 
                              epochs=5)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
5/5 [==============================] - 234s 47s/step - loss: 0.8665 - accuracy: 0.3233 - val_loss: 4.7928 - val_accuracy: 0.2188
Epoch 2/5
3/5 [=================>............] - ETA: 45s - loss: 0.5481 - accuracy: 0.5797

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet', include_top=False)
newOutput=base_model.output
newOutput=GlobalAveragePooling2D()(newOutput)
newOutput=Dense(1024,activation='relu')(newOutput) 
newOutput=Dense(1024,activation='relu')(newOutput) 
newOutput=Dense(512,activation='relu')(newOutput)
preds=Dense(num_classes,activation='softmax')(newOutput)


model=Model(inputs=base_model.input,outputs=preds)

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True
history=model.fit_generator(generator=train_generator,
                    validation_data= validation_generator, validation_steps=validation_steps,
                   steps_per_epoch=train_steps,
                   epochs=15)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
# VGG19 
from tensorflow.keras.applications.vgg19 import VGG19
base_model = VGG19(weights='imagenet' , include_top=False)
newOutput=base_model.output
newOutput=GlobalAveragePooling2D()(newOutput)
newOutput=Dense(1024,activation='relu')(newOutput) 
newOutput=Dense(1024,activation='relu')(newOutput) 
newOutput=Dense(512,activation='relu')(newOutput)

preds=Dense(num_classes,activation='softmax')(newOutput)


model=Model(inputs=base_model.input,outputs=preds)
for layer in model.layers[:5]:
    layer.trainable=False
for layer in model.layers[5:]:
    layer.trainable=True
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

history=model.fit_generator(generator=train_generator,
                    validation_data= validation_generator, validation_steps=validation_steps,
                   steps_per_epoch=train_steps,
                   epochs=5)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from keras.constraints import unit_norm
base_model = VGG19(weights='imagenet' , include_top=False)
newOutput=base_model.output
newOutput=GlobalAveragePooling2D()(newOutput)
newOutput=Dense(1024,activation='relu',kernel_constraint=unit_norm())(newOutput) 
newOutput=Dense(1024,activation='relu',kernel_constraint=unit_norm())(newOutput) 
newOutput=Dense(512,activation='relu',kernel_constraint=unit_norm())(newOutput)

preds=Dense(num_classes,activation='softmax')(newOutput)


model=Model(inputs=base_model.input,outputs=preds)
for layer in model.layers[:5]:
    layer.trainable=False
for layer in model.layers[5:]:
    layer.trainable=True
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

history=model.fit_generator(generator=train_generator,
                    validation_data= validation_generator, validation_steps=validation_steps,
                   steps_per_epoch=train_steps,
                   epochs=5)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False)
newOutput=base_model.output
newOutput=GlobalAveragePooling2D()(newOutput)
newOutput=Dense(1024,activation='relu')(newOutput) 
newOutput=Dense(1024,activation='relu')(newOutput) 
newOutput=Dense(512,activation='relu')(newOutput)

preds=Dense(num_classes,activation='softmax')(newOutput)
model=Model(inputs=base_model.input,outputs=preds)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

history=model.fit_generator(generator=train_generator,
                    validation_data= validation_generator, validation_steps=validation_steps,
                   steps_per_epoch=train_steps,
                   epochs=5)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
# mobilenet
base_model=MobileNet(weights='imagenet',include_top=False) 
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(num_classes,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)

In [ ]:
opt=Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit_generator(train_generator,steps_per_epoch=train_steps,
                              validation_data= validation_generator, 
                              validation_steps=validation_steps, 
                              epochs=5)

In [ ]:
## if every layers is trainable: there will be overfitting
from keras.constraints import unit_norm
base_model=MobileNet(weights='imagenet',include_top=False) 
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(num_classes,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
opt=Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit_generator(train_generator,steps_per_epoch=train_steps,
                              validation_data= validation_generator, 
                              validation_steps=validation_steps, 
                              epochs=EPOCHS)

In [ ]:
history = model.fit_generator(train_generator,steps_per_epoch=train_steps,
                              validation_data= validation_generator, 
                              validation_steps=validation_steps, 
                              epochs=EPOCHS)

In [ ]:
history = model.fit_generator(train_generator,steps_per_epoch=train_steps,
                              validation_data= validation_generator, 
                              validation_steps=validation_steps, 
                              epochs=EPOCHS)

## 7. Train the predictive model

### This is relative to the size of the data set and the compute resources you use. May take from minutes to hours

In [ ]:
history = model.fit_generator(train_generator,steps_per_epoch=train_steps,
                              validation_data= validation_generator, 
                              validation_steps=validation_steps, 
                              epochs=EPOCHS)


## 8. Take the original data and predict based on the new model

In [ ]:
# Reshape original input data images for predicting
img_check=datagen.flow_from_dataframe(dataframe=ndf, 
                                             x_col='id',
                                             y_col='label',
                                             target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=BS,
                                                 class_mode='categorical',
                                              
                                                 shuffle=True)

predictionsMade = []
preds = model.predict(img_check)
prediction_confidence = []
for i in range(0, len(preds)):
    prediction_confidence.append(np.amax(preds[i]))   



# Run all data through the prediction model that was created
for i in range (0,len(img_check)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)

print(prediction_confidence)    
# Count how many correct predictions were made
correct = 0
for i in range (0,len(predictionsMade)):
    if(predictionsMade[i] == yset[i]):
        correct += 1 
        
printmd("<b><span style='color:red'>Accuracy: " + str(correct/len(yset)) + "</span></b>")


## 9. Save the model file under models/, and reload it

In [ ]:
#Generate model file and save
modelName = user + "_cnn_" + out2.widget.result + "_" + str(epochNum.widget.result) + "_" + str(batchNum.widget.result) + ".h5"
modelPath = "models/"
if not os.path.exists(modelPath):
    os.makedirs(modelPath)
model.save(os.path.join(modelPath, modelName))
#Load model
from keras.models import load_model
model2 = load_model(os.path.join(modelPath, modelName))
printmd("<b><span style='color:red'>Model saved</span></b>")

## 10. Enter a new header for the prediction column

In [ ]:
# Translate back to original csv label names
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(yval_label[predictionsMade[i]])

from IPython.display import display
input_text = widgets.Text(
    value="predicted " + out2.widget.result,
    placeholder='Type label here',
    disabled=False
)
output_text = widgets.Text(
    value="predicted " + out2.widget.result,
    placeholder='New Header will be displayed here',
    disabled=False
)

def bind_input_to_output(sender):
    output_text.value = input_text.value

input_text.observe(bind_input_to_output)

print("Input new column Header Label: ")

display(input_text)
display(output_text)

## 11. Save the new version of CSV file, and give a name to new survey

In [ ]:
test_train_list

In [ ]:
# Append the new column w/ it's new column name
pred_conf_col = "pred_conf " + input_text.value
test_train_col = "test_train " + input_text.value

df[input_text.value] = finalPred
df[pred_conf_col] = prediction_confidence
df[test_train_col] = test_train_list

print(input_text.value)

new_file = absolutePath + csv_file[:-4]+'_v1.csv'
printmd("<b><span style='color:red'>A new temporary file will be created at: </span></b>")
print(new_file)
df.to_csv(new_file, index=None)



In [ ]:
#Input survey name

default_name = csv_file.split(".")[0] + "_" + str(EPOCHS) + "_" + str(BS) + "_" + str(im_dimension)

from IPython.display import display
input_text = widgets.Text(value=default_name)
output_text = widgets.Text(value=default_name)

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)

survey_name = output_text.value

In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

## 12. Generate the survey and create survey URL

In [ ]:
referer = survey_url.split("/main")[0] +"/"
upload_url = referer + "uploadCSV"
new_survey_url_base = survey_url.split(user)[0]

import requests
import re
csv = {"file": open(new_file, "rb")}
upload_data = {
    'name': input_text.value,
    'dzc': dzc_file,
    'user':user
}
headers = {
    'User-Agent': 'suave user agent',
    'referer': referer
}

r = requests.post(upload_url, files=csv, data=upload_data, headers=headers)

if r.status_code == 200:
    printmd("<b><span style='color:red'>New survey created successfully</span></b>")
    regex = re.compile('[^0-9a-zA-Z_]')
    s_url = survey_name
    s_url =  regex.sub('_', s_url)

    url = new_survey_url_base + user + "_" + s_url + ".csv" + "&views=" + views + "&view=" + view
    print(url)
    printmd("<b><span style='color:red'>Click the URL to open the new survey</span></b>")
else:
    printmd("<b><span style='color:red'>Error creating new survey. Check if a survey with this name already exists.</span></b>")
    printmd("<b><span style='color:red'>Reason: </span></b>"+ str(r.status_code) + " " + r.reason)

